In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df = pd.read_csv('dataset/cars.csv')

# Using multiple features: engine_cc and horsepower
X = df[['engine_cc', 'horsepower']].copy()
y = df['price_usd'].copy()

# Remove missing values
mask = ~(X.isna().any(axis=1) | y.isna())
X = X[mask]
y = y[mask]

print(f"Dataset size: {len(y)} samples")
print(f"Features: {list(X.columns)}")

In [ ]:
# Feature scaling
X_max = X.max()
y_max = y.max()

X = X / X_max
X = X.values

y = y / y_max
y = y.values.reshape(-1, 1)

In [ ]:
# Adding 1's column for intercept
n_rows = len(y)
X = np.c_[np.ones(n_rows), X]
print(X[:5])

# Initialize theta - now we have 3 parameters (intercept + 2 features)
theta = np.zeros((3, 1))

In [ ]:
# Gradient descent
epochs = 10000
learning_rate = 0.01
m = n_rows
costs = []

for i in range(epochs):
    h = X @ theta
    error = h - y
    cost = (1/(2*m)) * np.sum(np.square(error))
    costs.append(cost)
    
    gradient = (1/m) * (X.T @ error)
    theta = theta - learning_rate * gradient

print(f"Final theta: {theta.flatten()}")

In [ ]:
# Cost convergence plot
plt.figure(figsize=(8,5))
plt.plot(range(epochs), costs)
plt.title('Cost function convergence')
plt.xlabel('epochs')
plt.ylabel('cost')
plt.grid(True)
plt.show()

In [ ]:
# Interpret coefficients (converting back to original scale)
intercept_original = y_max * theta[0].item()
coef_engine = (y_max * theta[1].item()) / X_max['engine_cc']
coef_hp = (y_max * theta[2].item()) / X_max['horsepower']

print("\nMODEL EQUATION:")
print(f"price = {intercept_original:.2f} + {coef_engine:.4f}*engine_cc + {coef_hp:.4f}*horsepower")
print(f"\nIntercept: ${intercept_original:,.2f}")
print(f"Engine CC coefficient: ${coef_engine:.4f} per cc")
print(f"Horsepower coefficient: ${coef_hp:.4f} per hp")

In [ ]:
# Evaluation: MSE, RMSE, R²
y_pred = (X @ theta) * y_max
y_actual = y * y_max

# MSE = (1/m) * Σ(y_actual - y_pred)²
mse = (1/m) * np.sum(np.square(y_actual - y_pred))

# RMSE = √MSE
rmse = np.sqrt(mse)

# R² = 1 - (SS_res / SS_tot)
y_mean = np.mean(y_actual)
ss_res = np.sum(np.square(y_actual - y_pred))
ss_tot = np.sum(np.square(y_actual - y_mean))
r_squared = 1 - (ss_res / ss_tot)

print("\nEVALUATION METRICS:")
print(f"MSE: {mse:,.2f}")
print(f"RMSE: ${rmse:,.2f}")
print(f"R²: {r_squared:.6f} ({r_squared*100:.2f}%)")

In [ ]:
# Visualization: Predicted vs Actual
plt.figure(figsize=(8,6))
plt.scatter(y_actual, y_pred, alpha=0.5)
plt.plot([y_actual.min(), y_actual.max()], 
         [y_actual.min(), y_actual.max()], 
         'r--', linewidth=2)
plt.xlabel('Actual Price')
plt.ylabel('Predicted Price')
plt.title('Predicted vs Actual Prices')
plt.grid(True)
plt.show()

# Part C: Multiple Linear Regression
## Building a multiple linear regression model using mathematical formulas and gradient descent

In this notebook, we'll:
1. Build a multiple linear regression model using two or more features
2. Implement gradient descent manually (no sklearn)
3. Interpret coefficients
4. Evaluate using MSE, RMSE, and R²

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

## Step 1: Load and Prepare Data

In [ ]:
# Load dataset
df = pd.read_csv('dataset/cars.csv')

# Select multiple features for prediction
# We'll predict price based on engine_cc and horsepower
X = df[['engine_cc', 'horsepower']].copy()
y = df['price_usd'].copy()

# Remove any rows with missing values
mask = ~(X.isna().any(axis=1) | y.isna())
X = X[mask]
y = y[mask]

print(f"Dataset size: {len(y)} samples")
print(f"Features: {list(X.columns)}")
print(f"\nFeature statistics:")
print(X.describe())
print(f"\nTarget statistics:")
print(y.describe())

## Step 2: Feature Scaling
Normalize features to ensure gradient descent converges properly

In [ ]:
# Store original max values for later interpretation
X_max = X.max()
y_max = y.max()

print(f"Original feature max values:")
print(X_max)
print(f"\nOriginal target max value: {y_max}")

# Normalize features by dividing by max
X_scaled = X / X_max
y_scaled = y / y_max

# Convert to numpy arrays
X_scaled = X_scaled.values
y_scaled = y_scaled.values.reshape(-1, 1)

print(f"\nScaled X shape: {X_scaled.shape}")
print(f"Scaled y shape: {y_scaled.shape}")

## Step 3: Add Intercept Term
Add a column of ones to represent the intercept (θ₀)

In [ ]:
# Adding column of 1's for intercept term
# This allows us to write: y = X @ theta
# where theta = [θ₀, θ₁, θ₂, ...]

n_rows = len(y_scaled)
X_with_intercept = np.c_[np.ones(n_rows), X_scaled]

print(f"X with intercept shape: {X_with_intercept.shape}")
print(f"\nFirst 5 rows of X with intercept:")
print(X_with_intercept[:5])

# Initialize theta (parameters) - one for intercept + one for each feature
n_features = X_with_intercept.shape[1]
theta = np.zeros((n_features, 1))
print(f"\nInitial theta shape: {theta.shape}")

## Step 4: Implement Gradient Descent
Train the model using gradient descent algorithm

**Mathematical formulas:**
- Hypothesis: h(x) = X @ θ
- Cost function: J(θ) = (1/2m) * Σ(h(x) - y)²
- Gradient: ∇J = (1/m) * X^T @ (h(x) - y)
- Update rule: θ = θ - α * ∇J

In [ ]:
# Hyperparameters
epochs = 10000
learning_rate = 0.01
m = n_rows  # Number of training examples

# Store cost history for visualization
costs = []

# Gradient Descent Loop
for i in range(epochs):
    # Hypothesis: h(x) = X @ theta
    h = X_with_intercept @ theta
    
    # Error between prediction and actual
    error = h - y_scaled
    
    # Cost function: J(θ) = (1/2m) * sum((h(x) - y)²)
    cost = (1/(2*m)) * np.sum(np.square(error))
    costs.append(cost)
    
    # Gradient: (1/m) * X^T @ error
    gradient = (1/m) * (X_with_intercept.T @ error)
    
    # Update theta: θ = θ - α * gradient
    theta = theta - learning_rate * gradient
    
    # Print progress every 1000 epochs
    if (i + 1) % 1000 == 0:
        print(f"Epoch {i+1}/{epochs}, Cost: {cost:.6f}")

print(f"\nFinal cost: {costs[-1]:.6f}")
print(f"\nFinal theta (scaled):")
print(theta)

## Step 5: Visualize Cost Convergence

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(range(epochs), costs, linewidth=2)
plt.title('Cost Function Convergence', fontsize=14, fontweight='bold')
plt.xlabel('Epochs', fontsize=12)
plt.ylabel('Cost J(θ)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.show()

print(f"Initial cost: {costs[0]:.6f}")
print(f"Final cost: {costs[-1]:.6f}")
print(f"Cost reduction: {(costs[0] - costs[-1]):.6f}")

## Step 6: Interpret Coefficients
Convert scaled coefficients back to original scale for interpretation

In [ ]:
# Extract coefficients
theta_0 = theta[0].item()  # Intercept (scaled)
theta_1 = theta[1].item()  # Coefficient for engine_cc (scaled)
theta_2 = theta[2].item()  # Coefficient for horsepower (scaled)

print("=" * 60)
print("MULTIPLE LINEAR REGRESSION MODEL COEFFICIENTS")
print("=" * 60)
print(f"\nModel equation (scaled):")
print(f"price = {theta_0:.6f} + {theta_1:.6f}*engine_cc + {theta_2:.6f}*horsepower")

# Convert back to original scale for interpretation
# Since we scaled by max: y_scaled = y/y_max and X_scaled = X/X_max
# Original equation: y = intercept + coef1*x1 + coef2*x2
# Scaled equation: y/y_max = theta0 + theta1*(x1/x1_max) + theta2*(x2/x2_max)
# Therefore: y = y_max*theta0 + (y_max*theta1/x1_max)*x1 + (y_max*theta2/x2_max)*x2

intercept_original = y_max * theta_0
coef_engine_cc = (y_max * theta_1) / X_max['engine_cc']
coef_horsepower = (y_max * theta_2) / X_max['horsepower']

print(f"\nModel equation (original scale):")
print(f"price_usd = {intercept_original:.2f} + {coef_engine_cc:.4f}*engine_cc + {coef_horsepower:.4f}*horsepower")

print(f"\n" + "=" * 60)
print("COEFFICIENT INTERPRETATION")
print("=" * 60)
print(f"\nIntercept (θ₀): ${intercept_original:,.2f}")
print(f"  → Base price when all features are zero")

print(f"\nEngine CC Coefficient (θ₁): ${coef_engine_cc:.4f}")
print(f"  → For every 1 cc increase in engine size,")
print(f"    price increases by ${coef_engine_cc:.4f} (holding horsepower constant)")

print(f"\nHorsepower Coefficient (θ₂): ${coef_horsepower:.4f}")
print(f"  → For every 1 unit increase in horsepower,")
print(f"    price increases by ${coef_horsepower:.4f} (holding engine_cc constant)")
print("=" * 60)

## Step 7: Make Predictions and Calculate Evaluation Metrics
Calculate MSE, RMSE, and R² using mathematical formulas

In [ ]:
# Make predictions on scaled data
y_pred_scaled = X_with_intercept @ theta

# Convert predictions back to original scale
y_pred = y_pred_scaled * y_max
y_actual = y_scaled * y_max

# Calculate evaluation metrics manually

# 1. Mean Squared Error (MSE)
# MSE = (1/m) * Σ(y_actual - y_pred)²
mse = (1/m) * np.sum(np.square(y_actual - y_pred))

# 2. Root Mean Squared Error (RMSE)
# RMSE = √MSE
rmse = np.sqrt(mse)

# 3. R² (Coefficient of Determination)
# R² = 1 - (SS_res / SS_tot)
# SS_res = Σ(y_actual - y_pred)² (residual sum of squares)
# SS_tot = Σ(y_actual - y_mean)² (total sum of squares)
y_mean = np.mean(y_actual)
ss_res = np.sum(np.square(y_actual - y_pred))
ss_tot = np.sum(np.square(y_actual - y_mean))
r_squared = 1 - (ss_res / ss_tot)

print("=" * 60)
print("MODEL EVALUATION METRICS")
print("=" * 60)
print(f"\nMean Squared Error (MSE): ${mse:,.2f}²")
print(f"  → Average squared difference between predicted and actual prices")

print(f"\nRoot Mean Squared Error (RMSE): ${rmse:,.2f}")
print(f"  → Average prediction error in original units (USD)")
print(f"  → On average, predictions are off by ${rmse:,.2f}")

print(f"\nR² Score: {r_squared:.6f} ({r_squared*100:.4f}%)")
print(f"  → Model explains {r_squared*100:.4f}% of the variance in price")
print(f"  → Closer to 1.0 means better fit")

if r_squared > 0.8:
    print(f"  → This is a STRONG model (R² > 0.8)")
elif r_squared > 0.6:
    print(f"  → This is a MODERATE model (0.6 < R² < 0.8)")
else:
    print(f"  → This is a WEAK model (R² < 0.6)")

print("=" * 60)

## Step 8: Visualize Results - Predicted vs Actual

In [ ]:
plt.figure(figsize=(12, 5))

# Subplot 1: Predicted vs Actual scatter plot
plt.subplot(1, 2, 1)
plt.scatter(y_actual, y_pred, alpha=0.5, s=30)
plt.plot([y_actual.min(), y_actual.max()], 
         [y_actual.min(), y_actual.max()], 
         'r--', linewidth=2, label='Perfect Prediction')
plt.xlabel('Actual Price (USD)', fontsize=11)
plt.ylabel('Predicted Price (USD)', fontsize=11)
plt.title('Predicted vs Actual Prices', fontsize=13, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)

# Subplot 2: Residuals plot
residuals = y_actual - y_pred
plt.subplot(1, 2, 2)
plt.scatter(y_pred, residuals, alpha=0.5, s=30)
plt.axhline(y=0, color='r', linestyle='--', linewidth=2)
plt.xlabel('Predicted Price (USD)', fontsize=11)
plt.ylabel('Residuals (Actual - Predicted)', fontsize=11)
plt.title('Residual Plot', fontsize=13, fontweight='bold')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Residual statistics:")
print(f"Mean residual: ${np.mean(residuals).item():,.2f}")
print(f"Std residual: ${np.std(residuals).item():,.2f}")

## Step 9: 3D Visualization of Regression Plane

In [ ]:
# Create 3D plot showing the regression plane
fig = plt.figure(figsize=(14, 10))
ax = fig.add_subplot(111, projection='3d')

# Original data points
ax.scatter(X_scaled[:, 0], X_scaled[:, 1], y_scaled, 
           c='blue', marker='o', alpha=0.5, s=20, label='Actual Data')

# Create mesh for regression plane
x1_range = np.linspace(X_scaled[:, 0].min(), X_scaled[:, 0].max(), 30)
x2_range = np.linspace(X_scaled[:, 1].min(), X_scaled[:, 1].max(), 30)
x1_mesh, x2_mesh = np.meshgrid(x1_range, x2_range)

# Calculate predicted values for the mesh
y_mesh = theta[0] + theta[1] * x1_mesh + theta[2] * x2_mesh

# Plot regression plane
ax.plot_surface(x1_mesh, x2_mesh, y_mesh, 
                alpha=0.3, cmap='viridis', label='Regression Plane')

ax.set_xlabel('Engine CC (scaled)', fontsize=11, labelpad=10)
ax.set_ylabel('Horsepower (scaled)', fontsize=11, labelpad=10)
ax.set_zlabel('Price (scaled)', fontsize=11, labelpad=10)
ax.set_title('Multiple Linear Regression - 3D Visualization', 
             fontsize=14, fontweight='bold', pad=20)
ax.legend()

plt.tight_layout()
plt.show()

print("The plot shows:")
print("- Blue points: Actual data points")
print("- Colored surface: Regression plane fitted by our model")
print("- The plane represents: price = θ₀ + θ₁*engine_cc + θ₂*horsepower")

## Step 10: Example Predictions

In [ ]:
# Make some example predictions
print("=" * 60)
print("EXAMPLE PREDICTIONS")
print("=" * 60)

# Example 1
example_engine_cc = 2000
example_horsepower = 150
example_scaled = np.array([[1, 
                           example_engine_cc / X_max['engine_cc'], 
                           example_horsepower / X_max['horsepower']]])
prediction_scaled = example_scaled @ theta
prediction = prediction_scaled.item() * y_max

print(f"\nExample 1:")
print(f"  Engine CC: {example_engine_cc} cc")
print(f"  Horsepower: {example_horsepower} hp")
print(f"  Predicted Price: ${prediction:,.2f}")

# Example 2
example_engine_cc = 3500
example_horsepower = 250
example_scaled = np.array([[1, 
                           example_engine_cc / X_max['engine_cc'], 
                           example_horsepower / X_max['horsepower']]])
prediction_scaled = example_scaled @ theta
prediction = prediction_scaled.item() * y_max

print(f"\nExample 2:")
print(f"  Engine CC: {example_engine_cc} cc")
print(f"  Horsepower: {example_horsepower} hp")
print(f"  Predicted Price: ${prediction:,.2f}")

# Example 3
example_engine_cc = 1500
example_horsepower = 100
example_scaled = np.array([[1, 
                           example_engine_cc / X_max['engine_cc'], 
                           example_horsepower / X_max['horsepower']]])
prediction_scaled = example_scaled @ theta
prediction = prediction_scaled.item() * y_max

print(f"\nExample 3:")
print(f"  Engine CC: {example_engine_cc} cc")
print(f"  Horsepower: {example_horsepower} hp")
print(f"  Predicted Price: ${prediction:,.2f}")

print("=" * 60)

## Summary

In this notebook, we successfully:

1. **Built a multiple linear regression model** from scratch using gradient descent
2. **Used mathematical formulas** instead of libraries like sklearn
3. **Trained the model** on multiple features (engine_cc and horsepower)
4. **Interpreted coefficients** in both scaled and original units
5. **Evaluated the model** using:
   - **MSE** (Mean Squared Error): Measures average squared error
   - **RMSE** (Root Mean Squared Error): Error in original units
   - **R²** (Coefficient of Determination): Proportion of variance explained
6. **Visualized results** with scatter plots, residuals, and 3D regression plane

### Key Mathematical Concepts Used:
- **Hypothesis function**: h(x) = θ₀ + θ₁x₁ + θ₂x₂
- **Cost function**: J(θ) = (1/2m) * Σ(h(x) - y)²
- **Gradient descent**: θ := θ - α * ∇J(θ)
- **Matrix form**: h(x) = X @ θ
- **Gradient**: ∇J = (1/m) * X^T @ (h(x) - y)